# [WITCOM 2024 - Puerto Vallarta, MX 🇲🇽🌊🤖](https://witcom.upiita.ipn.mx/)

👋🏽👋🏽 ¡Buen día! En esta pequeña demostración vamos a explorar el uso de modelos YANG para interactuar con nuestros dispositivos de red.

## Qué haremos en esta demostración
- 🐍 Explorar la biblioteca de Python ncclient para interactuar vía NETCONF con un router Cisco
- ⚙️ Explorar las capacidades del dispositivo
- 🏗️ Explorar la estructura de un modelo de datos
- ⚡️ Leer y modificar configuraciones del dispositivo

Nuestro caso de uso van a ser las ```Listas de Control de Accesso```, o ```ACLs (Access Control Lists)```, que son configuraciones de seguridad que nos ayudar a filtar tipos muy específicos de tráfico en nuestros dispositivos de Capa 2 y Capa 3.

Digamos que queremos configurar una ACL similar a la siguiente, pero usando NETCONF + otras herramientas de automatización:

***
```
RP/0/RP0/CPU0:iosxr(config)# ipv4 access-list test-witcom-acl
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# remark ACL de prueba WITCOM
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# permit tcp 192.168.1.0 host 10.0.0.1 eq 80
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# permit tcp any any eq 22
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# deny udp any any eq 53
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# deny ip any any
RP/0/RP0/CPU0:iosxr(config-ipv4-acl)# commit
```
***

Lo que hace esta ligeramente compleja ACL es lo siguiente:

- Permite tráfico HTTP (puerto 80) desde 192.168.1.0 hacia 10.0.0.1.
- Permite tráfico SSH (puerto 22)
- Niega cualquier tráfico DNS (UDP/53)
- Niega todo el resto del tráfico.


### ⚙️ 1 - Explorando qué es lo que puede hacer mi router con sus modelos
Para esta demostración vamos a ocupar un Router Cisco de la familia IOSXR que está disponible 24/7 en el [servicio de sandbox de DevNet](https://developer.cisco.com/docs/iosxr-python/sandbox/#ios-xr-sandboxes), además de la biblioteca de Python ```ncclient``` para el manejo de NETCONF.

In [2]:
from ncclient import manager
import re

IOSXR_HOST = 'sandbox-iosxr-1.cisco.com'      # FQDN de nuestro router
IOSXR_PORT = 830                              # Puerto NETCONF (por defecto es 830)
IOSXR_USERNAME = 'admin'                      # Usuario
IOSXR_PASSWORD = 'C1sco12345'                 # Password

Iniciemos ahora una sesión SSH con nuestro router.

In [15]:
xr = manager.connect(
    host=IOSXR_HOST,
    port=IOSXR_PORT,
    username=IOSXR_USERNAME,
    password=IOSXR_PASSWORD,
    hostkey_verify=False,
    allow_agent=False,
    look_for_keys=False
)

🚀🚀 ¡Todo listo! Podemos hablar ahora con nuestro router usando NETCONF ...

Lo primero que sucede cuando establecemos esta sesión es el envío de una lista con todos los nmodelos YANG que nuestro router puede manejar. Estos modelos hacen cosas diferentes, tal y como configuración de paramétros operacionales, servicios, valores operativos, etc.

In [19]:
for capability in xr.server_capabilities:
    print(capability)

urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:capability:confirmed-commit:1.1
urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:interleave:1.0
http://cisco.com/ns/yang/Cisco-IOS-XR-um-event-manager-policy-map-cfg?module=Cisco-IOS-XR-um-event-manager-policy-map-cfg&revision=2021-06-16
http://cisco.com/ns/yang/Cisco-IOS-XR-ip-mobileip-cfg?module=Cisco-IOS-XR-ip-mobileip-cfg&revision=2019-04-05
http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-ma-oper?module=Cisco-IOS-XR-ipv4-ma-oper&revision=2019-10-25
http://cisco.com/ns/yang/Cisco-IOS-XR-lpts-ifib-oper?module=Cisco-IOS-XR-lpts-ifib-oper&revision=2020-06-24
http://cisco.com/ns/yang/Cisco-IOS-XR-um-router-ospfv3-cfg?module=Cisco-IOS-XR-um-router-ospfv3-cfg&revision=2021-02-12
http://cisco.com/ns/yang/Cisco-IOS-XR-tunnel-ip-ea-oper?module=Cisco-IO

En efecto son muchos ... Sin embargo, para encontrar el modelo que configura las ACLs, podemos simplemente usar regex y filtrar la lista.

In [13]:
for c in xr.server_capabilities:
    model = re.search('module=([^&]*acl[^&]*)&', c)
    if model is not None:
        print(model.group(1))

Cisco-IOS-XR-ipv4-acl-datatypes
Cisco-IOS-XR-ipv6-acl-datatypes
Cisco-IOS-XR-ipv4-acl-oper
Cisco-IOS-XR-ipv6-acl-cfg
Cisco-IOS-XR-es-acl-datatypes
Cisco-IOS-XR-common-acl-datatypes
Cisco-IOS-XR-es-acl-cfg
Cisco-IOS-XR-es-acl-oper
Cisco-IOS-XR-ipv4-acl-cfg
Cisco-IOS-XR-ipv6-acl-oper
openconfig-acl
cisco-xr-openconfig-acl-deviations


El modelo YANG que vamos a usar es ```Cisco-IOS-XR-ipv4-acl-cfg```. Los otros archivos tienen diferentes funciones, tal y como definiciones de estructuras de datos, configuraciones auxiliares, e incluso OpenConfig - que es una iniciativa Open Source para unificar los modelos de datos sin importar el fabricante (más información de este tema [aquí](https://github.com/ponchotitlan/oss-summit-OpenConfig-demo)).

Los modelos de datos igual suelen ser publicados online por casi todos los fabricantes. En nuestro caso, el modelo de ACLs para Cisco IOSXR está igual disponible [en este repositorio público](https://github.com/YangModels/yang/blob/main/vendor/cisco/xr/2411/Cisco-IOS-XR-ipv4-acl-cfg.yang).

### 🏗️ 2 - Hablando YANG con mi router

Vamos ahora a explorar el contenido de este modelo de datos para las ACLs:

In [20]:
ACL_NATIVE_MODULE = 'Cisco-IOS-XR-ipv4-acl-cfg'

data_model_payload = xr.get_schema(ACL_NATIVE_MODULE)
print (data_model_payload.data)

module Cisco-IOS-XR-ipv4-acl-cfg {

  /*** NAMESPACE / PREFIX DEFINITION ***/

  namespace "http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg";


  prefix "ipv4-acl-cfg";

  /*** LINKAGE (IMPORTS / INCLUDES) ***/

  import ietf-inet-types { prefix "inet"; }

  import Cisco-IOS-XR-types { prefix "xr"; }

  import cisco-semver { prefix "semver"; }

  import Cisco-IOS-XR-ipv4-acl-datatypes { prefix "dt1"; }

  include Cisco-IOS-XR-ipv4-ace-cfg {
    revision-date 2021-03-17;
  }

  /*** META INFORMATION ***/

  organization "Cisco Systems, Inc.";

  contact
    "Cisco Systems, Inc.
     Customer Service

     Postal: 170 West Tasman Drive
     San Jose, CA 95134

     Tel: +1 800 553-NETS

     E-mail: cs-yang@cisco.com";

  description 
    "This module contains a collection of YANG definitions
     for Cisco IOS-XR ipv4-acl package configuration.

     This module contains definitions
     for the following management objects:
       ipv4-acl-and-prefix-list: IPv4 ACL configuration dat

A veces, no es fácil leer un modelo de datos con la sintaxis en YANG, especialmente cuando el modelo es extenso y tiene muchas estructuras. La herramienta ```pyang``` permite la visualización de la estructura de forma gráfica.

En este ejemplo, vamos a guardar nuestro modelo de datos en un archivo y luego generar el gráfico para una mejor visualización. (Ojo, que vamos a necesitar usar también otros archivos que son dependencias).

In [21]:
ACL_YANG_DEPENDENCIES = ['Cisco-IOS-XR-types', 'cisco-semver', 'Cisco-IOS-XR-ipv4-acl-datatypes', 'Cisco-IOS-XR-ipv4-ace-cfg']

for dependency in ACL_YANG_DEPENDENCIES:
    with open(f"{dependency}.yang", "w") as f:
        f.write(xr.get_schema(dependency).data)

with open(f"{ACL_NATIVE_MODULE}.yang", "w") as f:
    f.write(data_model_payload.data)
    
!pyang -f tree Cisco-IOS-XR-ipv4-acl-cfg.yang

module: Cisco-IOS-XR-ipv4-acl-cfg
  +--rw ipv4-acl-and-prefix-list
     +--rw accesses
     |  +--rw access* [access-list-name]
     |     +--rw access-list-entries
     |     |  +--rw access-list-entry* [sequence-number]
     |     |     +--rw sequence-number             dt1:Ipv4-acl-sequence-number-range-acl
     |     |     +--rw grant?                      ipv4-acl-dt:Ipv4-acl-grant-enum
     |     |     +--rw protocol-operator?          ipv4-acl-dt:Ipv4-acl-operator-enum
     |     |     +--rw protocol?                   ipv4-acl-dt:Ipv4-acl-protocol-number
     |     |     +--rw protocol2?                  ipv4-acl-dt:Ipv4-acl-protocol-number
     |     |     +--rw source-network
     |     |     |  +--rw source-address?          inet:ipv4-address-no-zone
     |     |     |  +--rw source-wild-card-bits?   inet:ipv4-address-no-zone
     |     |     |  +--rw source-prefix-length?    xr:Ipv4-prefix-length
     |     |     +--rw destination-network
     |     |     |  +--rw destinati

¿Qué significa la simbología a la derecha de cada elemento?:

-	```?``` indica que el elemento es opcional
-	```*``` indica una lista o múltiples instancias (es decir, el elemento puede aparecer cero o más veces).
-	```!``` representa una elección (solo una de las opciones disponibles puede ser seleccionada).

### ⚡️ 3 - Leer & configurar usando este modelo de datos

Vamos ahora a filtrar de las configuraciones actuales (base de datos ```running```) las ACLs existentes


In [16]:
acl_filter = """
<filter>
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
  </ipv4-acl-and-prefix-list>
</filter>
"""
my_acls = xr.get_config(source='running', filter=(acl_filter))
print(my_acls)

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:b4134680-1ac0-4b8d-9004-971a17e79ed3" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <data>
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
   <accesses>
    <access>
     <access-list-name>acl_1</access-list-name>
     <access-list-entries>
      <access-list-entry>
       <sequence-number>10</sequence-number>
       <remark>Block-Denial-of-Service</remark>
       <sequence-str>10</sequence-str>
      </access-list-entry>
      <access-list-entry>
       <sequence-number>100</sequence-number>
       <grant>deny</grant>
       <source-network>
        <source-address>192.0.2.10</source-address>
       </source-network>
       <sequence-str>100</sequence-str>
      </access-list-entry>
      <access-list-entry>
       <sequence-number>200</sequence-number>
       <grant>deny</grant>
       <source-network>
        <source-address>192.0.2.11<

Podemos ahora ocupar este payload de ejemplo para crear nuestra propia configuración, añadiendo parámetros un poco más complejos.

Vamos entonces a hacer commit de esta configuración pero en la base de datos ```candidate``` del router. De esta manera podemos validar la integridad de nuestro XML. Posteriormente podríamos hacer commit hacia la base de datos ```running```.

In [18]:
new_acl = '''
<config>
  <ipv4-acl-and-prefix-list xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ipv4-acl-cfg">
    <accesses>
      <access>
        <access-list-name>test_witcom_ACL</access-list-name>
        <access-list-entries>

          <access-list-entry>
            <sequence-number>10</sequence-number>
            <remark>ACL de prueba WITCOM</remark>
          </access-list-entry>
          
          <!-- permit tcp 192.168.1.0 host 10.0.0.1 eq 80 -->
          <access-list-entry>
            <sequence-number>20</sequence-number>
            <grant>permit</grant>
            <protocol>6</protocol>
            <source-network>
              <source-address>192.168.1.0</source-address>
            </source-network>
            <destination-network>
              <destination-address>10.0.0.1</destination-address>
            </destination-network>
            <destination-port>
              <destination-operator>equal</destination-operator>
              <first-destination-port>80</first-destination-port>
            </destination-port>
          </access-list-entry>
          
          <!-- permit tcp any any eq 22 -->
          <access-list-entry>
            <sequence-number>30</sequence-number>
            <grant>permit</grant>
            <protocol>6</protocol>
            <destination-port>
              <destination-operator>equal</destination-operator>
              <first-destination-port>22</first-destination-port>
            </destination-port>
          </access-list-entry>
          
          <!-- deny udp any any eq 53 -->
          <access-list-entry>
            <sequence-number>40</sequence-number>
            <grant>deny</grant>
            <protocol>17</protocol>
            <destination-port>
              <destination-operator>equal</destination-operator>
              <first-destination-port>53</first-destination-port>
            </destination-port>
          </access-list-entry>
          
          <!-- deny ip any any -->
          <access-list-entry>
            <sequence-number>50</sequence-number>
            <grant>deny</grant>
            <protocol>0</protocol>
          </access-list-entry>
          
        </access-list-entries>
      </access>
    </accesses>
  </ipv4-acl-and-prefix-list>
</config>
'''

test = xr.edit_config(new_acl, target='candidate', format='xml')
print(test)

<?xml version="1.0"?>
<rpc-reply message-id="urn:uuid:d45623ff-cc86-4379-9ca8-5a8fb710bafb" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
 <ok/>
</rpc-reply>



***
🇲🇽🌊🤖 WITCOM 2024